In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.externals import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../sample_csv_data/ohport_multi_device_downsampling_flow.csv')
df = df.drop(columns=['Unnamed: 0'])

In [4]:
df.head()

,timeStampFirst,timeStampLast,duration,linkProtocol,networkProtocol,transportProtocol,applicationProtocol,srcMac,dstMac,srcIP,dstIP,srcPort,dstPort,packetCount,totalPayloadByte,appplicationPayloadByte,iot,device,dns,dhcp-server,http,ntp,https,smtp-ssl,imap-ssl,ssdp,icslap,stm-pproc,stun,ws-discovery,upnp-evnt,xmpp,android,dstPort25050,dstPort49152,dstPort49153,dstPort49154,wellKnownDstPort,ephemeralDstPort,registedDstPort,srcPort49152,srcPort49153
0,1475798997877,1475798997897,20,Ethernet,IPv4,TCP,NaN,ec:1a:59:83:28:11,00:16:6c:ab:6b:88,192.168.1.193,192.168.1.249,4892.0,49152.0,5,524,0,True,Belkin wemo motion sensor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1475782011005,1475783763656,1752651,Ethernet,IPv4,TCP,NaN,ec:1a:59:83:28:11,ec:1a:59:79:f4:89,192.168.1.193,192.168.1.223,3603.0,49153.0,14,1306,0,True,Belkin wemo motion sensor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1475812199234,1475812199277,43,Ethernet,IPv4,TCP,NaN,ec:1a:59:83:28:11,ec:1a:59:79:f4:89,192.168.1.193,192.168.1.223,4576.0,49153.0,7,653,0,True,Belkin wemo motion sensor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1475089201043,1475089201057,14,Ethernet,IPv4,TCP,NaN,ec:1a:59:83:28:11,ec:1a:59:79:f4:89,192.168.1.193,192.168.1.223,3165.0,49153.0,7,653,0,True,Belkin wemo motion sensor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1475531144213,1475531144264,51,Ethernet,IPv4,TCP,NaN,ec:1a:59:83:28:11,ec:1a:59:79:f4:89,192.168.1.193,192.168.1.223,4015.0,49152.0,7,653,0,True,Belkin wemo motion sensor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
# one hot for network & transport protocol
one_hot = pd.get_dummies(df['networkProtocol'])
df = df.join(one_hot)
one_hot = pd.get_dummies(df['transportProtocol'])
df = df.join(one_hot)

# device name to number
device_list = df.device.unique()
print(device_list)
device_dict = dict(zip(device_list, range(len(device_list))))
df = df.replace({'device': device_dict})

['Belkin wemo motion sensor' 'Belkin Wemo switch' 'Samsung SmartCam'
 'Amazon Echo' 'Insteon Camera' 'Light Bulbs LiFX Smart Bulb'
 'Withings Smart Baby Monitor' 'Netatmo Welcome'
 'Withings Aura smart sleep sensor' 'Netatmo weather station'
 'TP-Link Day Night Cloud camera' 'PIX-STAR Photo-frame' 'HP Printer'
 'Triby Speaker' 'Not IOT']


In [44]:
categoricals = [
    'dns',
    'dhcp-server',
    'http',
    'ntp',
    'https',
    'smtp-ssl',
    'imap-ssl',
    'ssdp',
    'icslap',
    'stm-pproc',
    'stun',
    'ws-discovery',
    'upnp-evnt',
    'xmpp',
    'android',
    'dstPort25050',
    'dstPort49152',
    'dstPort49153',
    'dstPort49154',
    'srcPort49152',
    'srcPort49153',
    'ephemeralDstPort', 
    'registedDstPort',
    'wellKnownDstPort',
    'IPv4',
    'IPv6',
    'TCP',
    'UDP',
]
continuous = ['duration', 'packetCount', 'totalPayloadByte', 'appplicationPayloadByte']


X_con = df[continuous].values[:,:]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_con)
X_cat = df[categoricals].values[:,:]
X = np.hstack([X_scaled, X_cat])
Y = df[['device']].values[:]
Y = Y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [45]:
models = [
    {
        'model': DecisionTreeClassifier(random_state = 100),
        'name': 'Decision Tree',
        'active': True
    },
    {
        'model': KNeighborsClassifier(n_neighbors=3),
        'name': 'KNN',
        'active': True
    },
    {
        'model': GaussianNB(),
        'name': 'GaussianNB',
        'active': True
    },
    {
        'model':  RandomForestClassifier(n_estimators=100),
        'name': 'Random Forest',
        'active': True
    },
    {
        'model': AdaBoostClassifier(n_estimators=100),
        'name': 'Ada Boosting Decision Tree',
        'active': True
    },
    {
        'model': GradientBoostingClassifier(n_estimators=100),
        'name': 'Gradient Boosting Decision Tree',
        'active': True
    }
]

In [46]:
for m in models:
    if m['active'] == True:
        m['model'].fit(X_train, y_train)
        y_pred = m['model'].predict(X_test)
        print(m['name'], "accurency is", accuracy_score(y_test,y_pred), "\nf1 score is", f1_score(y_test, y_pred, average=None), "\n")

Decision Tree accurency is 0.9397777777777778 
f1 score is [0.97709924 0.98850575 0.97689769 0.95932203 0.94493783 0.96333333
 0.94527363 0.94462541 0.93918919 0.94138544 0.96397942 0.85493827
 0.96434635 0.9024     0.82857143] 



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNN accurency is 0.9126666666666666 
f1 score is [0.98470948 0.99022801 0.95008052 0.93132328 0.78303426 0.95065789
 0.87755102 0.91465378 0.89965398 0.94903339 0.9775475  0.80952381
 0.9527027  0.91680261 0.78776291] 

GaussianNB accurency is 0.4533333333333333 
f1 score is [0.71518193 0.89036545 0.73387097 0.         0.         0.27358491
 0.43906511 0.39185751 0.         0.53080569 0.52       0.42313324
 0.45057471 0.32432432 0.20606061] 



/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Random Forest accurency is 0.9482222222222222 
f1 score is [0.98769231 0.99019608 0.97385621 0.95711835 0.9430605  0.96482412
 0.91666667 0.95098039 0.92748735 0.97183099 0.97586207 0.8673947
 0.97       0.93569132 0.89084507] 



/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Ada Boosting Decision Tree accurency is 0.19666666666666666 
f1 score is [0.05995204 0.34380776 0.11178615 0.00883652 0.00617284 0.30985915
 0.07142857 0.23101777 0.02179837 0.8039604  0.40875912 0.05979073
 0.20353982 0.38698011 0.08196721] 



/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Gradient Boosting Decision Tree accurency is 0.938 
f1 score is [0.97846154 0.98861789 0.97217676 0.9556314  0.94793537 0.96160267
 0.90572391 0.94444444 0.91554054 0.95454545 0.97746967 0.86427457
 0.96795953 0.8917609  0.84536082] 



In [47]:
features = [
    'dns',
    'dhcp-server',
    'http',
    'ntp',
    'https',
    'smtp-ssl',
    'imap-ssl',
    'ssdp',
    'icslap',
    'stm-pproc',
    'stun',
    'ws-discovery',
    'upnp-evnt',
    'xmpp',
    'android',
    'dstPort25050',
    'dstPort49152',
    'dstPort49153',
    'dstPort49154',
    'srcPort49152',
    'srcPort49153',
    'ephemeralDstPort', 
    'registedDstPort',
    'wellKnownDstPort',
    'IPv4',
    'IPv6',
    'TCP',
    'UDP',
    'duration', 
    'packetCount', 
    'totalPayloadByte', 
    'appplicationPayloadByte'
]

sum = 0
i = 0
for im in models[3]['model'].feature_importances_:
    print(str(im*100), features[i])
    sum += im
    i += 1

22.060005490916385 dns
10.984656196263344 dhcp-server
28.72682538583987 http
4.540239859653254 ntp
1.3081968508965145 https
0.6176250374862898 smtp-ssl
3.613609820273569 imap-ssl
2.0586542404206503 ssdp
3.7720483968485254 icslap
0.6604951659404279 stm-pproc
0.07090262171267885 stun
0.09326196179805 ws-discovery
0.03678462147874759 upnp-evnt
0.1021353263101254 xmpp
1.0736109490712575 android
0.2213696074824344 dstPort25050
0.23196705603780965 dstPort49152
0.731947593298944 dstPort49153
0.4095156884609869 dstPort49154
3.930170735977638 srcPort49152
1.4715007728023095 srcPort49153
1.072694419825821 ephemeralDstPort
0.1685792980994901 registedDstPort
1.7062558886961867 wellKnownDstPort
2.2373592434593923 IPv4
2.3947554453982383 IPv6
1.7850437618167083 TCP
0.0 UDP
0.40384977021155133 duration
0.40264657981069535 packetCount
1.3936451422881424 totalPayloadByte
1.7196470714239445 appplicationPayloadByte


In [40]:
print(sum)

1.0
